In [ ]:
import pandas as pd
import numpy as np
#from post_tag import *
import csv

In [ ]:
translations = pd.read_csv("training.txt", sep="\t", header=None, names=['labels','phrases'], quoting=csv.QUOTE_NONE)
translations

In [ ]:
#Notes about the data:
# it's about 50/50 labels



In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import sklearn
#from stop_words import get_stop_words
from time import time
from nltk.corpus import stopwords


In [ ]:
es_stop = stopwords.words('spanish')

In [ ]:
M_doc_freq = 0.80
min_doc_freq = 10

CV = CountVectorizer(max_df=M_doc_freq, min_df=min_doc_freq, stop_words=es_stop, binary=True, lowercase=True)

In [ ]:
# Fit the vectorizer
t0 = time()
CV = CV.fit(translations['phrases'].values)

phrase_matrix = CV.transform(translations['phrases'].values)
print("done in %0.3fs." % (time() - t0))
tf_feature_names = CV.get_feature_names()

In [ ]:
n_wrds=len(CV.vocabulary_)

In [ ]:
cols = ['bw' + str(i) for i in range(n_wrds)]

BofW_feats = pd.DataFrame(phrase_matrix.todense(), columns=cols)
BofW_feats.to_csv('BoW_feats.csv')

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import csv

In [ ]:
data = phrase_matrix
target = translations['labels'].values
classf =  SVC()

In [ ]:
t0 = time()
classf.fit(data,target)
print "Score of SVM: " + str(classf.score(data,target))
print("done in %0.3fs." % (time() - t0))

In [ ]:
t0 = time()
svd = TruncatedSVD(n_components=100, random_state=42)
svd.fit(data) 
data = svd.transform(data)
print("SVD fit done in %0.3fs." % (time() - t0))

In [ ]:
t0 = time()
data = data
pca = PCA(n_components =500)
pca.fit(data)
data = pca.transform(data)
print("PCA fit done in %0.3fs." % (time() - t0))

In [ ]:
t0 = time()
classf = SVC()
scores = cross_validation.cross_val_score(classf,data,target, n_jobs=-1, cv = 10)
print "Scores: " + str(scores)
print "Crossval Final score: "+ str(sum(scores)/10)
print("Cross val done in %0.3fs." % (time() - t0))

In [ ]:
# How to process a test set and predict the labels 

test = pd.read_csv("example_test.txt", sep="\t", header=None, names=['labels','phrases'], lineterminator='\n', quoting=csv.QUOTE_NONE)
test

In [ ]:
testlabs = classf.predict(svd.transform(CV.transform(test['phrases'])))

In [ ]:
test.labels = test['labels'] = testlabs
test

In [ ]:
test.to_csv("example_test_output.txt", sep="\t", header=False, index=False, quoting=csv.QUOTE_NONE)

In [ ]:
test.iloc[0]['phrases']

In [ ]:
import pandas as pd
import numpy as np
from post_tag import *
import csv
from time import time
translations = pd.read_csv("training.txt", sep="\t", header=None, names=['labels','phrases'], quoting=csv.QUOTE_NONE)



In [ ]:
# play around with pos tagging
trans = translations['phrases'].apply(postag)

In [ ]:
test = pd.read_csv("example_test.txt", sep="\t", header=None, names=['labels','phrases'], lineterminator='\n', quoting=csv.QUOTE_NONE)

test

In [ ]:
t0 = time()
test_tagged = test['phrases'].apply(postag)
print("done in %0.3fs."%(time()-t0))

In [ ]:
fdist = itertools.permutations(words, 2)
permu_names = [f for f in fdist]
tags_DF = pd.DataFrame(np.asmatrix(list(test_tagged)),columns = permu_names)

test_new = pd.concat([test, tags_DF], axis=1,join_axes=[test.index])
test_new.drop('phrases',axis=1, inplace=True)
test_new.to_csv(path_or_buf='pos_tags_train_data.csv')

In [ ]:
import cProfile
cProfile.run('test_tagged = test["phrases"].apply(postag)')

In [ ]:
translat = translations[0:100]

In [ ]:
tags_DF

In [ ]:
# Now transform all the data
import pandas as pd
import numpy as np
from post_tag import *
import csv
from time import time
translations = pd.read_csv("training.txt", sep="\t", header=None, names=['labels','phrases'], quoting=csv.QUOTE_NONE)
fdist = itertools.permutations(words, 2)
permu_names = [f for f in fdist]



In [ ]:
trans = translat['phrases'].apply(postag)
tags_DF = pd.DataFrame(np.asmatrix(list(trans)),columns = permu_names)

test_new = pd.concat([translat, tags_DF], axis=1,join_axes=[translat.index])
test_new.drop('phrases',axis=1, inplace=True)
test_new.to_csv(path_or_buf='pos_tags_train_data.csv')